# 오차역전파

> 2.2.6 장에 해당하는 코드

In [1]:
import torch
import matplotlib.pyplot as plt

## 미분 계산하기

In [2]:
# 코드 2-15

def function(x):
    """원래 함수"""
    return x**2

def prime_function(x):
    """도함수"""
    return 2*x

x0 = torch.FloatTensor([4])
y0 = prime_function(x0)
print("differentiation of x={} is {:.1f}".format(x0.item(), y0.item()))

differentiation of x=4.0 is 8.0


## 매개변수 개수 구하기

In [3]:
# 코드 2-16

import torch.nn as nn

torch.manual_seed(70)

class Network(nn.Module):
    """XOR Network"""
    def __init__(self, input_size, hidden_size, output_size):
        super(Network, self).__init__()
        # 층을 구성
        # input layer > hidden layer 
        self.linear_ih = nn.Linear(in_features=input_size, out_features=hidden_size)
        # hidden layer > output layer
        self.linear_ho = nn.Linear(in_features=hidden_size, out_features=output_size)
        # activation layer
        self.activation_layer = nn.Sigmoid()
        
    def forward(self, x):
        z1 = self.linear_ih(x)
        a1 = self.activation_layer(z1)
        z2 = self.linear_ho(a1)
        y = self.activation_layer(z2)
        return y

net = Network(input_size=2, hidden_size=2, output_size=1)
num_params = 0
for p in net.parameters():
    num_params += p.view(-1).size(0)
print(num_params)

9


## 계산그래프

### z 함수 순방향 전파

In [4]:
# 코드 2-17

def z_function(t):
    return t**2

def t_function(x, y):
    return x + y

def forward(x, y):
    t = t_function(x, y)
    z = z_function(t)
    return z

x = torch.Tensor([1])
y = torch.Tensor([3])
z = forward(x, y)
print(z)

tensor([16.])


### z 함수 역방향 전파

In [5]:
# 코드 2-18

def z_prime(t):
    """dz/dt"""
    return 2*t

def t_prime():
    """dt/dx = dt/dy"""
    return 1

def backward(t):
    """dz/dx"""
    dx = z_prime(t) * t_prime()
    return dx

x = torch.Tensor([1])
y = torch.Tensor([3])
t = t_function(x, y)
dx = backward(t)
print(dx)

tensor([8.])


### 역전파 수행과정

In [6]:
# 코드 2-19

x = torch.Tensor([1]).requires_grad_()
y = torch.Tensor([3])
z = forward(x, y)

# gradient가 필요한지 점검
print("requires grad: x={} y={}, z={}".format(
    x.requires_grad, y.requires_grad, z.requires_grad))
print("gradient function of z: {}".format(z.grad_fn))

# 역전파 수행 버튼
z.backward()
print("dx = {}".format(x.grad))

requires grad: x=True y=False, z=True
gradient function of z: <PowBackward0 object at 0x7f1869b0b860>
dx = tensor([8.])


# 오차역전파

In [7]:
# 코드 2-20

import torch.optim as optim

# 입력과 타겟텐서 생성
x = torch.Tensor([[0, 0], [0, 1], [1, 0], [1, 1]])
t = torch.Tensor([0, 1, 1, 0])

# 이전에 만든 XOR 네트워크 정의
net = Network(input_size=2, hidden_size=2, output_size=1)

# 손실함수 정의
loss_function = nn.BCELoss()

# 경사하강법 optimizer 정의: 모델의 매개변수를 전달해줘야한다.
optimizer = optim.SGD(params=net.parameters(), lr=0.5)

# 몇회 학습할지 결정: STEP/EPOCH
STEP = 10001

# 학습과정
for step in range(STEP):
    # 경사 초기화
    net.zero_grad()
    # 순방향 전파
    y = net(x)
    # 손실값 계산
    loss = loss_function(y.squeeze(), t)
    # . backward() 함수를 호출하여 역방향 전파
    loss.backward()
    # .step() 함수를 호출하여 경사하강법으로 매개변수 업데이트
    optimizer.step()
    if step % 1000 == 0:
        print("{}:\t {:.4f}".format(step, loss.item()))
        
# 올바른 정답을 출력하는지 테스트한다 
pred = net(x).ge(0.5)
print(pred.view(-1))

0:	 0.6927
1000:	 0.2574
2000:	 0.0210
3000:	 0.0102
4000:	 0.0067
5000:	 0.0049
6000:	 0.0039
7000:	 0.0032
8000:	 0.0028
9000:	 0.0024
10000:	 0.0021
tensor([0, 1, 1, 0], dtype=torch.uint8)
